<목차>
---
1. AutoEncoder의 개념  
2. MNIST 데이터 재구축
3. MNIST 분류기 구현  
  - 파인 튜닝과 전이 학습
  - MNIST 분류기 구현

# 1. AutoEncoder의 개념
---
- 비지도 학습을 위한 인공신경망 구조  
- 오토인코더의 출력은 원본 데이터를 **재구축**(Reconstruction)한 결과  
- 은닉층의 출력값은 원본 데이터에서 불필요한 특징들을 제거한 **압축된 특징**

# 2. MNIST 데이터 재구축
---

In [ ]:
!pip install tensorflow==1.2

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#데이터 다운로드
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
#학습에 필요한 설정값 정의
learning_rate = 0.02
training_epochs = 20
batch_size = 256
display_step = 1 #손실함수 출력 주기
examples_to_show = 10 #보여줄 재구축 이미지
input_size = 784
hidden1_size = 256
hidden2_size = 128

In [18]:
#입력값 받기
x = tf.placeholder(tf.float32, shape=[None, input_size])

#-------------------------------------모델 정의--------------------------------------------
def build_autoencoder(x):
  #인코딩 784 > 256 > 128
  W1 = tf.Variable(tf.random_normal(shape=[input_size, hidden1_size]))
  b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H1_output = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

  W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]))
  b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]))
  H2_output = tf.nn.sigmoid(tf.matmul(H1_output,W2) + b2)
  
  #디코딩 128 > 256 > 784
  W3 = tf.Variable(tf.random_normal(shape=[hidden2_size, hidden1_size]))
  b3 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H3_output = tf.nn.sigmoid(tf.matmul(H2_output,W3) + b3)

  W4 = tf.Variable(tf.random_normal(shape=[hidden1_size, input_size]))
  b4 = tf.Variable(tf.random_normal(shape=[input_size]))
  reconstructed_x = tf.nn.sigmoid(tf.matmul(H3_output,W4) + b4)

  return reconstructed_x

#---------------------------------그래프 구조 생성--------------------------------
y_pred = build_autoencoder(x)

#-------------------------손실함수와 옵티마이저 정의------------------------------
loss = tf.reduce_mean(tf.pow(x - y_pred, 2))
train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

#------------------------------------그래프 실행-----------------------------------
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  #최적화 반복
  for epoch in range(training_epochs):
    #전체 배치 불러오기
    total_batch = int(mnist.train.num_examples / batch_size)
    #각 배치에 최적화 수행
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, current_loss = sess.run([training_epochs, loss], feed_dict={x:batch_xs})

    #학습결과 출력
    if epoch % display_step == 0:
      print("반복: %d, 손실함수: %f" % ((epoch+1), current_loss))

  
  #test데이터 재구축
  reconstructed_result = sess.run(y_pred, feed_dict={x:mnist.test.images[:examples_to_show]})

  #test데이터 vs 재구축 결과
  f, a = plt.subplots(2, 10, figsize=(10, 2))
  for i in range(examples_to_show):
    a[0][i].imshow(np.reshape(mnist.test.images[i], (28,28)))
    a[1][i].imshow(np.reshape(reconstructed_result[i], (28,28)))
  f.savfig('reconstructed_mnist_image.png')
  f.show()
  plt.draw()
  plt.waitforbuttonpress()

# 3. MNIST 분류기 구현
---
- 파인튜닝(Fine-Tuning)과 전이학습(Transfer learning)
  - B 문제를 풀기 위한 모델을 학습시키고자 할 때, A 문제를 풀기 위해 학습된 파라미터를 가져와서 수정하는 기법
  - AutoEncoder의 디코딩 부분을 삭제하고 인코딩된 특징값을 분류기의 입력값으로 학습  
  - 분류 성능을 더 높이는 효과
- 분류기 학습 순서  
1) **사전 학습**: 데이터 재구축 목적으로 AutoEncoder 학습  
2) **파인 튜닝**: 숫자 분류를 목적으로 다시 최적화하기 위해 AutoEncoder 학습

In [3]:
#학습에 필요한 설정값 정의
learning_rate_RMSProp = 0.02
learning_rate_GradientDescent = 0.5 
num_epochs = 100
batch_size = 256
display_step = 1
input_size = 784
hidden1_size = 128
hidden2_size = 64 
#설정값 아래 코드에 직접 대입

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, 10]) #실제 MNIST 숫자값

#--------------------------------AutoEncoder 구조 정의-----------------------------------------------
def build_autoencoder(x):
  #인코딩 784 > 256 > 128
  W1 = tf.Variable(tf.random_normal(shape=[input_size, hidden1_size]))
  b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H1_output = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

  W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]))
  b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]))
  H2_output = tf.nn.sigmoid(tf.matmul(H1_output,W2) + b2)
  
  #디코딩 128 > 256 > 784
  W3 = tf.Variable(tf.random_normal(shape=[hidden2_size, hidden1_size]))
  b3 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H3_output = tf.nn.sigmoid(tf.matmul(H2_output,W3) + b3)

  W4 = tf.Variable(tf.random_normal(shape=[hidden1_size, input_size]))
  b4 = tf.Variable(tf.random_normal(shape=[input_size]))
  reconstructed_x = tf.nn.sigmoid(tf.matmul(H3_output,W4) + b4)

  return reconstructed_x, H2_output

#------------------------------Softmax 분류기 정의------------------------------------------------------
def build_softmax_classifier(x):
  
  #원본 이미지(784) 대신 재구축된 특징(64)을 입력값으로 받음
  W_softmax = tf.Variable(tf.zeros([hidden2_size, 10])) 
  b_softmax = tf.Variable(tf.zeros([10]))
  y_pred = tf.nn.softmax(tf.matmul(x, W_softmax) + b_softmax)

  return y_pred

#----------------------------------그래프 구조 생성------------------------------------------------------
y_pred, extracted_features = build_autoencoder(x)
y_pred_softmax = build_softmax_classifier(extracted_features)

#----------------------------------1) 사전 학습----------------------------------------------------------
pretraining_loss = tf.reduce_mean(tf.pow(x - y_pred, 2))
pretraining_train_step = tf.train.RMSPropOptimizer(learning_rate_RMSProp).minimize(pretraining_loss)

#-------------------------2) 파인 튜닝(손실함수와 옵티마이저 정의)---------------------------------------
finetuning_loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_pred_softmax), reduction_indices=[1]))
finetuning_train_step = tf.train.GradientDescentOptimizer(learning_rate_GradientDescent).minimize(finetuning_loss)

#----------------------------------------그래프 실행-----------------------------------------------------
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  total_batch = int(mnist.train.num_examples/batch_size)

  #사전학습 최적화
  for epoch in range(num_epochs):
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, pretraining_loss_print = sess.run([pretraining_train_step, pretraining_loss], feed_dict={x:batch_xs})
    if epoch % display_step == 0:
      print("반복: %d, 사전학습 손실함수: %f" %((epoch+1), pretraining_loss_print))
  print("사전학습 최적화 완료")

  #파인튜닝 최적화
  for epoch in range(num_epochs+100):
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, finetuning_loss_print = sess.run([finetuning_train_step, finetuning_loss], feed_dict={x:batch_xs, y:batch_ys})
    if epoch % display_step == 0:
      print("반복: %d, 파인튜닝 손실함수: %f" %((epoch+1), finetuning_loss_print))
  print("파인튜닝 최적화 완료")

  #정확도 출력
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_pred_softmax, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_predicion, tf.float32))
  print("정확도: %f" %sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))


반복: 1, 사전학습 손실함수: 0.173456
반복: 2, 사전학습 손실함수: 0.107189
반복: 3, 사전학습 손실함수: 0.094686
반복: 4, 사전학습 손실함수: 0.080522
반복: 5, 사전학습 손실함수: 0.067303
반복: 6, 사전학습 손실함수: 0.063146
반복: 7, 사전학습 손실함수: 0.059718
반복: 8, 사전학습 손실함수: 0.057705
반복: 9, 사전학습 손실함수: 0.054638
반복: 10, 사전학습 손실함수: 0.051806
반복: 11, 사전학습 손실함수: 0.050090
반복: 12, 사전학습 손실함수: 0.049157
반복: 13, 사전학습 손실함수: 0.046456
반복: 14, 사전학습 손실함수: 0.047479
반복: 15, 사전학습 손실함수: 0.047068
반복: 16, 사전학습 손실함수: 0.045666
반복: 17, 사전학습 손실함수: 0.042720
반복: 18, 사전학습 손실함수: 0.041650
반복: 19, 사전학습 손실함수: 0.040096
반복: 20, 사전학습 손실함수: 0.040642
반복: 21, 사전학습 손실함수: 0.040900
반복: 22, 사전학습 손실함수: 0.040152
반복: 23, 사전학습 손실함수: 0.040299
반복: 24, 사전학습 손실함수: 0.039190
반복: 25, 사전학습 손실함수: 0.036930
반복: 26, 사전학습 손실함수: 0.036297
반복: 27, 사전학습 손실함수: 0.034307
반복: 28, 사전학습 손실함수: 0.033592
반복: 29, 사전학습 손실함수: 0.033702
반복: 30, 사전학습 손실함수: 0.031986
반복: 31, 사전학습 손실함수: 0.032740
반복: 32, 사전학습 손실함수: 0.032521
반복: 33, 사전학습 손실함수: 0.031337
반복: 34, 사전학습 손실함수: 0.030314
반복: 35, 사전학습 손실함수: 0.032664
반복: 36, 사전학습 손실함수: 0.031706
반

오토인코더를 사용하면   
소프트맥스 분류기만 사용했을 때와 ANN을 이용했을 때보다 정확도가 더 향상된다